In [1]:
import numpy as np
import pandas as pd
import altair as alt
from altair_saver import save

The function below is a auxiliary function used to create the datasets 'chik_br_2010-2022_quarter.csv' and 'dengue_br_2010-2022_quarter.csv' if we have the datasets with the time series of cases separated by state.

In [2]:

def get_cases_by_year(filename):
    """
    Function to aggregate the time series of cases by quarter and compute the incidence based on the population
    
    Parameters
    ----------
    state: str
        two letter code of the state.

    Returns
    -------
    df
        a dataframe with cases and incidence by quarter

    """
    
    df = pd.read_parquet(filename, columns=['casos'])
    #df.index = pd.to_datetime(df.index)
    df = df.resample('Q').sum()
    
    
    df_pop = pd.read_csv('./data/poptcu2010-2022_rgi.csv', 
        usecols=[ 'UF',  'POP22', 'POP21',
       'POP20', 'POP19', 'POP18', 'POP17', 'POP16', 'POP15', 'POP14', 'POP13',
       'POP12', 'POP11', 'POP10' ])
    
    df_pop = df_pop.loc[df_pop.UF == state]

    df_pop = df_pop.groupby(['UF']).sum()

    df_pop['POP23'] = df_pop['POP22']
    
    df['pop'] = [df_pop[f'POP{str(x)[-2:]}'].values[0] for x in df.index.year]

    df['inc'] = df['casos']/df['pop']

    trimestre = []

    for x in df.index:
        if x.month == 3:

            trimestre.append(f'{x.year}01')
        if x.month ==6:

            trimestre.append(f'{x.year}02')
        if x.month == 9:

            trimestre.append(f'{x.year}03')

        if x.month == 12:

            trimestre.append(f'{x.year}04')


    df['trimestre'] = trimestre
    df['UF'] = state
    
    return df


The snippet of code below can be used to generate the dataset of cases and incidence by quarter for all the stats in Brazil: 

''' 

df_end = pd.DataFrame()

for state in estados.keys():
    
    filename = f'../infodengue_data/data/{state}_{AGRAVO}.parquet'
    
    df_end = pd.concat([df_end, get_cases_by_year(filename)])
    
df_end.head()
'''

In [3]:
def plot_heatmap_single(agravo = 'dengue'):
    
    """
    Function to plot a heatmap of the incidence by quarter
    
    Parameters
    ----------
    agravo: str
        name of the disease (dengue or chik)

    Returns
    -------
    fig
        an altair figura

    """
    
    # load the data
    df_end= pd.read_csv(f'./data/{agravo}_br_2010-2022_quarter.csv')
    df_end.set_index('data_iniSE', inplace = True)
    df_end.index = pd.to_datetime(df_end.index)

    # the column order is used to agg the states in the yaxis based on the region 
    df_end['order'] = np.nan

    df_end.loc[df_end.regiao == 'Sudeste', 'order'] = 1
    df_end.loc[df_end.regiao == 'Sul', 'order'] = 2
    df_end.loc[df_end.regiao == 'Nordeste', 'order'] = 3
    df_end.loc[df_end.regiao == 'Norte', 'order'] = 4
    df_end.loc[df_end.regiao == 'Centro-Oeste', 'order'] = 5

    # selection field to filter by region
    regiao_radio = alt.binding_radio(options=np.append([None],df_end['regiao'].unique()),labels=['All'], name="Selecione")
    
    selection = alt.selection_point(fields=['regiao'], bind=regiao_radio)

    if agravo=='dengue':
        title = 'Dengue'
        color_scheme = 'viridis'

    if agravo == 'chik':
        title='Chikungunya'
        color_scheme = 'orangered'

    # create the figure
    fig = alt.Chart(data= df_end, width=650, height=500, title=title).mark_rect(filled=True).encode(
    x= alt.X('trimestre-tick', type='nominal', title='Quarter', sort = alt.SortField('trimestre', order='ascending'),
                
                axis=alt.Axis(
            labelColor=alt.condition(f"datum.value[0] < 3",  alt.value('black'), alt.value('grey')))  # this line changes the color of x-axis ticks based on a condition   
                ),
        y = alt.Y('UF', title='State', sort = alt.SortField('order', order='ascending')), 
        tooltip = [ alt.Tooltip(field = "trimestre-tick", title = 'Quarter', type = "nominal"),
                    alt.Tooltip(field = "inc", title = 'Incidence by 100k notified', type = "quantitative")],
        color=alt.condition(selection, 
                            alt.Color('inc:Q', scale=alt.Scale(scheme=color_scheme),
                            legend=alt.Legend(direction='vertical', orient='left', legendY=30, title = None)),
                            alt.value('lightgray'))
    ).add_params(
        selection
    )
    
    return fig



In [6]:
fig = plot_heatmap_single(agravo = 'dengue')

fig

alt.Chart(...)

It's possible to save this map as a html

In [5]:
fig.save('files_html/heatmap_dengue.html')